In [3]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import os

In [4]:
spark = SparkSession.builder.master("local").appName("NaiveBayesJob").getOrCreate()


In [5]:
def load_data(file_path, file_name):
    full_path = os.path.join(file_path, file_name)
    return pd.read_csv(full_path)

In [6]:
def mapOceanProximity(proximity):

    if(proximity == "NEAR BAY"):
        return 1
    
    elif(proximity == "INLAND"):
        return 1
    
    elif(proximity =="<1H OCEAN"):
        return 0
    
    elif(proximity=="NEAR OCEAN"):
        return 1
    
    elif(proximity =="ISLAND"):
        return 1
    
    # else:
    #     return 0

In [7]:
def createLabeledPoint(fields):

    longitude = float(fields[0])
    latitude = float(fields[1])
    housing_median_age = float(fields[2])
    total_rooms = float(fields[3])
    total_bedrooms = float(fields[4])
    population = float(fields[4])
    households = float(fields[5])
    median_income = float(fields[6])
    median_house_value = float(fields[7])
    ocean_proximity = mapOceanProximity(fields[8])

    return LabeledPoint(ocean_proximity, np.array([longitude, latitude, housing_median_age, total_rooms,
                        total_bedrooms, population, households, median_income, median_house_value]))

In [8]:
DATA_PATH = os.path.join(os.getcwd(),'datasets', 'housing')
FILE_NAME = "housing.csv"
housing_data = load_data(DATA_PATH, FILE_NAME)


In [9]:
df = spark.read.csv(os.path.join(DATA_PATH, FILE_NAME), header=True, sep=",").na.fill(value=0, subset=["total_bedrooms", "longitude","latitude", "housing_median_age", "total_rooms", "population", "households", "median_income", "median_house_value", "ocean_proximity"])
housing_rdd = df.rdd

In [3]:
spark.stop()

NameError: name 'spark' is not defined

In [1]:
header = housing_rdd.first()
housing_rdd  = housing_rdd.filter(lambda x: x != header)
# housing_rdd = housing_rdd.map(lambda x: x.split(","))

training_data_rdd, test_data_rdd = housing_rdd.randomSplit(weights=[8.0,2.0], seed=1)  # raw_data_rdd.map(createLabeledPoint)




NameError: name 'housing_rdd' is not defined

In [11]:
train_rdd, test_rdd = training_data_rdd.map(createLabeledPoint), test_data_rdd.map(createLabeledPoint)


In [12]:
train_rdd

PythonRDD[16] at RDD at PythonRDD.scala:53

In [13]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils
model = NaiveBayes.train(train_rdd, 1.0)


21/07/02 16:07:41 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*ar

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (192.168.0.155 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/rdd.py",

In [ ]:
# command to drop rows that have null values
#test_df.na.drop(subset=["total_bedrooms", "median_income"]).show()
#test_df.show()
#test_df = df.filter(df.total_rooms.between(1255.0,1256))

AttributeError: 'RDD' object has no attribute 'show'

Signature: df.na.drop(how='any', thresh=None, subset=None)
Docstring:
Returns a new :class:`DataFrame` omitting rows with null values.
:func:`DataFrame.dropna` and :func:`DataFrameNaFunctions.drop` are aliases of each other.

.. versionadded:: 1.3.1

Parameters
----------
how : str, optional
    'any' or 'all'.
    If 'any', drop a row if it contains any nulls.
    If 'all', drop a row only if all its values are null.
thresh: int, optional
    default None
    If specified, drop rows that have less than `thresh` non-null values.
    This overwrites the `how` parameter.
subset : str, tuple or list, optional
    optional list of column names to consider.

Examples
--------
>>> df4.na.drop().show()
+---+------+-----+
|age|height| name|
+---+------+-----+
| 10|    80|Alice|
+---+------+-----+
File:      ~/anaconda3/envs/py_tf/lib/python3.7/site-packages/pyspark/sql/dataframe.py
Type:      method


In [11]:
!which python3

/usr/bin/python3
